In [ ]:
# Referenced from https://medium.com/@katia.ossetchkina57/absolute-beginners-guide-tweepy-and-the-twitter-apis-6070995efc63
# Libraries needed.
import os
import tweepy
import time
import pandas as pd
import json
from datetime import datetime

In [ ]:
#enter credentials
consumer_api_key = '' ## enter your own consumer API key 
consumer_api_secret = ''## enter your own consumer API Secret 

In [ ]:
callback_uri = 'oob' #This allows the passing of standard queries in requests.
auth = tweepy.OAuthHandler(consumer_api_key,consumer_api_secret, callback_uri) #Creates a connection to Twitter 
redirect_url = auth.get_authorization_url() #Creates a secure link.
print(redirect_url)

In [ ]:
# 1. Click the above link once created.
# 2. Authorize the app if you haven't already.
# 3. After authorization you will be directed to a pin.
    # Keep this pin for the next cell.

In [ ]:
# Enter pin in the input below and press enter.
user_pin = input('Provide your user pin from the link: ')
user_pin

In [ ]:
#Twitter search operators documentation: https://developer.twitter.com/en/docs/twitter-api/premium/search-api/overview#AvailableOperators
auth.get_access_token(user_pin) #Access token creation.
api = tweepy.API(auth) #Initializing of the API endpoint through Tweepy.

In [ ]:
returned_tweets=[]
for pages in tweepy.Cursor(api.search_full_archive, 
                          label='dev', 
                          query = " #purplereign has:images -is:retweet OR #gohuskies has:images -is:retweet OR #uwvwsu has:images -is:retweet OR #applecup has:images -is:retweet" ,
                          fromDate =201911290000,
                          toDate =201911292359, 
                          maxResults =500).pages(2):
    # page is a list of statuses
   returned_tweets.extend(pages)

In [ ]:
# Creates a table of all the Images URL in the query 

MediaURL=[]

for status in returned_tweets:
    if 'media' in status.entities:
        for image in status.entities['media']:
            MediaURL.append(image['media_url'])
    else: MediaURL.append('NA')
   

In [ ]:
for info in returned_tweets[:700]:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.text)
     print(info.source)
     print(info.place)
     print(info.user.screen_name)
     print(info.user.id)
     print(info.lang)
     print(info.quote_count)
     print(info.reply_count)
     print(info.favorite_count)
     print(info.entities)
     print("\n")

In [ ]:
# Create the data frame for our twitter data
from pandas import DataFrame

returned = [[ info.id, 
              info.created_at, 
              info.favorite_count, 
              info.retweet_count,
              info.reply_count,
              info.quote_count,
              info.text.encode("utf-8").decode("utf-8"),
              info.entities,
              info.user.screen_name,
              info.source,
              info.place,
              info.user.id,
              info.lang] 
             for idx,info in enumerate(returned_tweets)]
returned_df = DataFrame(returned,columns=["id","created_at","favorite_count","retweet_count", "Reply_Count","Quote Count","text", "entities","screen_name","Source","Place","User ID","Lang"])
returned_df.head()
print(returned_df)


In [ ]:
returned_df['MediaURL'] = MediaURL

In [ ]:
returned_df.to_csv('Downloaded_tweets_Full_search.csv', index=False, header=True)